In [17]:
#!pip install seaborn 

import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
import tqdm as tqdm
sns.__version__

'0.11.2'

In [18]:
FOLDER = 'data'
FILE = '20-years-odds.csv'


data = pd.read_csv(f'{FOLDER}/{FILE}')
LIMIT = int(data.shape[0]*0.85)

data.shape, LIMIT

((65960, 17), 56066)

In [19]:
data.head()

,home_team,away_team,correct_score,1N2_1,1N2_N,1N2_2,matchdate,period,location,season,competition,home_score,away_score,bet_won,target,odd_win,best_odd_won
0,Arsenal,Aston Villa,1:0,1.30,4.7,8.0,1999-05-16,1999-05,england,1999,premier-league-1998,1.0,0.0,1,0.0,1.30,True
1,Charlton,Sheffield Wed,0:1,1.65,3.6,4.0,1999-05-16,1999-05,england,1999,premier-league-1998,0.0,1.0,2,2.0,4.00,False
2,Chelsea,Derby,2:1,1.50,3.5,6.0,1999-05-16,1999-05,england,1999,premier-league-1998,2.0,1.0,1,0.0,1.50,True
3,Coventry,Leeds,2:2,3.00,2.9,2.4,1999-05-16,1999-05,england,1999,premier-league-1998,2.0,2.0,N,1.0,2.90,False
4,Liverpool,Wimbledon FC,3:0,1.55,3.3,5.8,1999-05-16,1999-05,england,1999,premier-league-1998,3.0,0.0,1,0.0,1.55,True


In [20]:
teams = [data.home_team.tolist()] + [data.away_team.tolist()]

teams = np.unique(teams[0])
len(np.unique(teams))

288

In [21]:
scores = np.unique(data.correct_score)
df_scores = pd.DataFrame(data.correct_score.value_counts(normalize=True))
#df_scores.columns = [['scores','ratio']]
#df_scores.sort_values(ascending=False)
df_scores.columns = ['ratio']
df_scores.sort_values('ratio',ascending=False)

,ratio
1:1,0.129518
1:0,0.108475
2:1,0.091343
0:0,0.084991
2:0,0.080033
...,...
9:1,0.000015
6:5,0.000015
4:7,0.000015
1:9,0.000015


# Baseline

Sinmply compute a baseline by taking the 1N2 odds, transpose them to 1/odds then scale date and train a multi-output regressor.

## Result

Depending on the choice of the estimator the score (accuracy) is between 41-48 % of good prediction about game outcome.
For the correct score, the prediction is around 11% of good result.

In [22]:
df = data[['1N2_1','1N2_N','1N2_2','home_score','away_score']]
df

,1N2_1,1N2_N,1N2_2,home_score,away_score
0,1.30,4.70,8.00,1.0,0.0
1,1.65,3.60,4.00,0.0,1.0
2,1.50,3.50,6.00,2.0,1.0
3,3.00,2.90,2.40,2.0,2.0
4,1.55,3.30,5.80,3.0,0.0
...,...,...,...,...,...
65955,6.85,4.03,1.53,2.0,2.0
65956,4.04,3.75,1.88,1.0,3.0
65957,1.29,5.53,10.81,4.0,0.0
65958,6.12,4.46,1.52,1.0,2.0


In [23]:
X = 1/df[['1N2_1','1N2_N','1N2_2']]
y = df.drop(['1N2_1','1N2_N','1N2_2'],axis=1)
# summarize dataset
print(X.shape, y.shape)

(65960, 3) (65960, 2)


In [24]:
X_train=X[:LIMIT]
X_test=X[LIMIT:]

y_train = y[:LIMIT]
y_test = y[LIMIT:]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(56066, 3) (9894, 3) (56066, 2) (9894, 2)


In [25]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
model = LinearRegression(fit_intercept=False)



model = DecisionTreeRegressor()
model = KNeighborsRegressor(n_neighbors=11)
model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=11)

In [27]:

yhat = model.predict(X_test)
eval = y_test.copy()

yhat49=yhat+0.4999
yhat49=yhat49.astype(int)

eval['home_pred']=yhat49[:,0]
eval['away_pred']=yhat49[:,1]

conditions = [
    ((eval['home_pred'] > eval['away_pred']) & (eval['home_score'] > eval['away_score'])),
    ((eval['home_pred'] == eval['away_pred']) & (eval['home_score'] == eval['away_score'])),
    ((eval['home_pred'] < eval['away_pred']) & (eval['home_score'] < eval['away_score']))
    
]
choices = [True,True,True]

eval['prediction_1N2'] = np.select(conditions, choices, default=False)


conditions = [
    ((eval['home_pred'] == eval['home_score']) & (eval['away_pred'] == eval['away_score']))
]
choices = [True]

eval['prediction_score'] = np.select(conditions, choices, default=False)


eval.prediction_1N2.value_counts(normalize=True),eval.prediction_score.value_counts(normalize=True)


(False    0.560441
 True     0.439559
 Name: prediction_1N2, dtype: float64,
 False    0.889125
 True     0.110875
 Name: prediction_score, dtype: float64)

In [28]:
eval

,home_score,away_score,home_pred,away_pred,prediction_1N2,prediction_score
56066,2.0,1.0,2,0,True,False
56067,1.0,2.0,2,1,False,False
56068,1.0,1.0,1,2,False,False
56069,1.0,1.0,2,1,False,False
56070,2.0,0.0,2,0,True,True
...,...,...,...,...,...,...
65955,2.0,2.0,1,2,False,False
65956,1.0,3.0,1,2,True,False
65957,4.0,0.0,2,1,True,False
65958,1.0,2.0,1,1,False,False


# Features Engineering

This section will add some features to the base line to see if we can improve it

In [29]:
data

,home_team,away_team,correct_score,1N2_1,1N2_N,1N2_2,matchdate,period,location,season,competition,home_score,away_score,bet_won,target,odd_win,best_odd_won
0,Arsenal,Aston Villa,1:0,1.30,4.70,8.00,1999-05-16,1999-05,england,1999,premier-league-1998,1.0,0.0,1,0.0,1.30,True
1,Charlton,Sheffield Wed,0:1,1.65,3.60,4.00,1999-05-16,1999-05,england,1999,premier-league-1998,0.0,1.0,2,2.0,4.00,False
2,Chelsea,Derby,2:1,1.50,3.50,6.00,1999-05-16,1999-05,england,1999,premier-league-1998,2.0,1.0,1,0.0,1.50,True
3,Coventry,Leeds,2:2,3.00,2.90,2.40,1999-05-16,1999-05,england,1999,premier-league-1998,2.0,2.0,N,1.0,2.90,False
4,Liverpool,Wimbledon FC,3:0,1.55,3.30,5.80,1999-05-16,1999-05,england,1999,premier-league-1998,3.0,0.0,1,0.0,1.55,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65955,Udinese,Juventus,2:2,6.85,4.03,1.53,2021-08-22,2021-08,italy,2022,serie-a-2021,2.0,2.0,N,1.0,4.03,False
65956,Empoli,Lazio,1:3,4.04,3.75,1.88,2021-08-21,2021-08,italy,2022,serie-a-2021,1.0,3.0,2,2.0,1.88,True
65957,Inter,Genoa,4:0,1.29,5.53,10.81,2021-08-21,2021-08,italy,2022,serie-a-2021,4.0,0.0,1,0.0,1.29,True
65958,Torino,Atalanta,1:2,6.12,4.46,1.52,2021-08-21,2021-08,italy,2022,serie-a-2021,1.0,2.0,2,2.0,1.52,True


In [30]:
data.query('home_team == "Arsenal" & away_team == "Aston Villa"')[['home_score','away_score']].sum()[1]

19.0

In [32]:
data

,home_team,away_team,correct_score,1N2_1,1N2_N,1N2_2,matchdate,period,location,season,competition,home_score,away_score,bet_won,target,odd_win,best_odd_won
0,Arsenal,Aston Villa,1:0,1.30,4.70,8.00,1999-05-16,1999-05,england,1999,premier-league-1998,1.0,0.0,1,0.0,1.30,True
1,Charlton,Sheffield Wed,0:1,1.65,3.60,4.00,1999-05-16,1999-05,england,1999,premier-league-1998,0.0,1.0,2,2.0,4.00,False
2,Chelsea,Derby,2:1,1.50,3.50,6.00,1999-05-16,1999-05,england,1999,premier-league-1998,2.0,1.0,1,0.0,1.50,True
3,Coventry,Leeds,2:2,3.00,2.90,2.40,1999-05-16,1999-05,england,1999,premier-league-1998,2.0,2.0,N,1.0,2.90,False
4,Liverpool,Wimbledon FC,3:0,1.55,3.30,5.80,1999-05-16,1999-05,england,1999,premier-league-1998,3.0,0.0,1,0.0,1.55,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65955,Udinese,Juventus,2:2,6.85,4.03,1.53,2021-08-22,2021-08,italy,2022,serie-a-2021,2.0,2.0,N,1.0,4.03,False
65956,Empoli,Lazio,1:3,4.04,3.75,1.88,2021-08-21,2021-08,italy,2022,serie-a-2021,1.0,3.0,2,2.0,1.88,True
65957,Inter,Genoa,4:0,1.29,5.53,10.81,2021-08-21,2021-08,italy,2022,serie-a-2021,4.0,0.0,1,0.0,1.29,True
65958,Torino,Atalanta,1:2,6.12,4.46,1.52,2021-08-21,2021-08,italy,2022,serie-a-2021,1.0,2.0,2,2.0,1.52,True


In [34]:
for idx, r in data.iterrows():
    t1 = r.home_team
    t2 = r.away_team
    ht = data.query('home_team == @t1 & away_team == @t2')
    ht = ht[['home_score','away_score']].sum()
    data.loc[idx,'tot_home_score']=ht[0]
    data.loc[idx,'tot_away_score']=ht[1]
    
data

    
    

,home_team,away_team,correct_score,1N2_1,1N2_N,1N2_2,matchdate,period,location,season,competition,home_score,away_score,bet_won,target,odd_win,best_odd_won,tot_home_score,tot_away_score
0,Arsenal,Aston Villa,1:0,1.30,4.70,8.00,1999-05-16,1999-05,england,1999,premier-league-1998,1.0,0.0,1,0.0,1.30,True,44.0,19.0
1,Charlton,Sheffield Wed,0:1,1.65,3.60,4.00,1999-05-16,1999-05,england,1999,premier-league-1998,0.0,1.0,2,2.0,4.00,False,10.0,11.0
2,Chelsea,Derby,2:1,1.50,3.50,6.00,1999-05-16,1999-05,england,1999,premier-league-1998,2.0,1.0,1,0.0,1.50,True,18.0,4.0
3,Coventry,Leeds,2:2,3.00,2.90,2.40,1999-05-16,1999-05,england,1999,premier-league-1998,2.0,2.0,N,1.0,2.90,False,9.0,12.0
4,Liverpool,Wimbledon FC,3:0,1.55,3.30,5.80,1999-05-16,1999-05,england,1999,premier-league-1998,3.0,0.0,1,0.0,1.55,True,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65955,Udinese,Juventus,2:2,6.85,4.03,1.53,2021-08-22,2021-08,italy,2022,serie-a-2021,2.0,2.0,N,1.0,4.03,False,15.0,35.0
65956,Empoli,Lazio,1:3,4.04,3.75,1.88,2021-08-21,2021-08,italy,2022,serie-a-2021,1.0,3.0,2,2.0,1.88,True,7.0,10.0
65957,Inter,Genoa,4:0,1.29,5.53,10.81,2021-08-21,2021-08,italy,2022,serie-a-2021,4.0,0.0,1,0.0,1.29,True,36.0,8.0
65958,Torino,Atalanta,1:2,6.12,4.46,1.52,2021-08-21,2021-08,italy,2022,serie-a-2021,1.0,2.0,2,2.0,1.52,True,19.0,26.0


In [35]:
df = data[['1N2_1','1N2_N','1N2_2','tot_home_score','tot_away_score','home_score','away_score']]
df

,1N2_1,1N2_N,1N2_2,tot_home_score,tot_away_score,home_score,away_score
0,1.30,4.70,8.00,44.0,19.0,1.0,0.0
1,1.65,3.60,4.00,10.0,11.0,0.0,1.0
2,1.50,3.50,6.00,18.0,4.0,2.0,1.0
3,3.00,2.90,2.40,9.0,12.0,2.0,2.0
4,1.55,3.30,5.80,6.0,1.0,3.0,0.0
...,...,...,...,...,...,...,...
65955,6.85,4.03,1.53,15.0,35.0,2.0,2.0
65956,4.04,3.75,1.88,7.0,10.0,1.0,3.0
65957,1.29,5.53,10.81,36.0,8.0,4.0,0.0
65958,6.12,4.46,1.52,19.0,26.0,1.0,2.0


In [36]:


X = df[['1N2_1','1N2_N','1N2_2','tot_home_score','tot_away_score']]
y = df.drop(['1N2_1','1N2_N','1N2_2','tot_home_score','tot_away_score'],axis=1)
# summarize dataset
print(X.shape, y.shape)

X_train=X[:LIMIT]
X_test=X[LIMIT:]

y_train = y[:LIMIT]
y_test = y[LIMIT:]

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

(65960, 5) (65960, 2)
(56066, 5) (9894, 5) (56066, 2) (9894, 2)


In [37]:
model = LinearRegression(fit_intercept=False)



model = DecisionTreeRegressor()
model = KNeighborsRegressor(n_neighbors=11)
model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=11)

In [38]:
yhat = model.predict(X_test)
eval = y_test.copy()

yhat49=yhat+0.4999
yhat49=yhat49.astype(int)

eval['home_pred']=yhat49[:,0]
eval['away_pred']=yhat49[:,1]

conditions = [
    ((eval['home_pred'] > eval['away_pred']) & (eval['home_score'] > eval['away_score'])),
    ((eval['home_pred'] == eval['away_pred']) & (eval['home_score'] == eval['away_score'])),
    ((eval['home_pred'] < eval['away_pred']) & (eval['home_score'] < eval['away_score']))
    
]
choices = [True,True,True]

eval['prediction_1N2'] = np.select(conditions, choices, default=False)


conditions = [
    ((eval['home_pred'] == eval['home_score']) & (eval['away_pred'] == eval['away_score']))
]
choices = [True]

eval['prediction_score'] = np.select(conditions, choices, default=False)


eval.prediction_1N2.value_counts(normalize=True),eval.prediction_score.value_counts(normalize=True)

(True     0.5
 False    0.5
 Name: prediction_1N2, dtype: float64,
 False    0.859713
 True     0.140287
 Name: prediction_score, dtype: float64)